In [1]:
%run 'vehicle_detection.py'

svc_file = "./classifier.p"
svc = pickle.load(open(svc_file, "rb" ))

scaler_file = "./scaler.p"
X_scaler = pickle.load(open(scaler_file, "rb" ))

'''
tranformer_file = "./tranformer.p"
X_transformer = pickle.load(open(tranformer_file, "rb" ))
'''

'''
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
'''
y_start_stop = [402, None]


test_images = glob.glob('../test_images/*.jpg')
for image_file in test_images:
    image = mpimg.imread(image_file).astype(np.float32)
    print('image.shape: ', image.shape)
    draw_image = np.copy(image)

    windows = slide_window(image, x_start_stop=Settings.x_start_stop, y_start_stop=Settings.y_start_stop,
                        xy_window=Settings.xy_window, xy_overlap=Settings.xy_overlap)

    hot_windows = search_windows(image, windows, svc, scaler=X_scaler, transformer=None, color_space=Settings.color_space, 
                            spatial_size=Settings.spatial_size, hist_bins=Settings.hist_bin, 
                            orient=Settings.orient, pix_per_cell=Settings.pix_per_cell, 
                            cell_per_block=Settings.cell_per_block, 
                            hog_channel=Settings.hog_channel, hog_resize=Settings.hog_resize)                       

    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)
    
    heat = np.zeros_like(image[:,:,0]).astype(np.float32)
    
    # Add heat to each box in box list
    heat = add_heat(heat, hot_windows)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat, 1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)

    window_img = window_img.astype(np.uint8)
    draw_img = draw_img.astype(np.uint8)
    fig = plt.figure(figsize=(12,9))
    plt.subplot(131)
    plt.imshow(window_img)
    plt.title('Hot Windows')
    plt.subplot(132)
    plt.imshow(draw_img)
    plt.title('Car Positions')
    plt.subplot(133)
    plt.imshow(heatmap, cmap='hot')
    plt.title('Heat Map')
    fig.tight_layout()

    #plt.imshow(window_img)
    plt.show()



/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


image.shape:  (720, 1280, 3)


TypeError: only length-1 arrays can be converted to Python scalars